In [1]:
!pip install OpenNMT-py
!pip install light-the-torch
!ltt install torch torchvision
!pip install transformers
!pip install bert-extractive-summarizer
!pip install transformers
!pip install bert-extractive-summarizer
!pip install tqdm

     |████████████████████████████████| 215kB 8.4MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 2.5MB 14.4MB/s 
     |████████████████████████████████| 276kB 56.6MB/s 
     |████████████████████████████████| 81kB 13.1MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 748.8MB 24kB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 1.2MB 53.8MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=005a0af40aa9dc3af108878a7f235aeafb1ba88b35b08ee83aa85af63bed2c9f
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for configargparse: filename=ConfigArgParse-1.2.3-cp36-none-any.whl size=19328 sha256=91aa0190b4ed05c1998219ca9d9683290a6d3436ce3729319a745e2966a83fba
  Stored in directory: /root/.cache/pip/wheels/bd/d6/53/034032da9498

In [32]:
from transformers import *
from summarizer import Summarizer
import pandas as pd
import re
import nltk
from tqdm._tqdm_notebook import tqdm_notebook
from nltk.tokenize.treebank import TreebankWordDetokenizer
tqdm_notebook.pandas()

nltk.download('punkt')
custom_config = AutoConfig.from_pretrained('cahya/bert-base-indonesian-1.5G')
custom_config.output_hidden_states = True
custom_tokenizer = AutoTokenizer.from_pretrained('cahya/bert-base-indonesian-1.5G')
custom_model = AutoModel.from_pretrained('cahya/bert-base-indonesian-1.5G',
                                          config=custom_config)
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

Downloading:   0%|          | 0.00/468 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [45]:
 def summarize_text(text, language="indon"):
     result = model(text)
     return result

In [51]:
df = pd.read_csv('/content/drive/MyDrive/articles.csv')
df['body_no_tags'] = df['body'].apply(lambda x: re.sub('<.*?>', '', x))
df = df[df['body_no_tags'].str.len() < 5000]
df['summarized_body'] = df['body_no_tags'].progress_apply(lambda x: summarize_text(x))

  0%|          | 0/3612 [00:00<?, ?it/s]

In [53]:
to_translate = df['body'].apply(lambda x: (" ").join(nltk.word_tokenize(x)))

In [54]:
to_translate.to_csv("translate.txt", sep="\t", header=False, index=False)

In [27]:
!onmt_translate -model /content/drive/MyDrive/model_step_125000.pt -src translate.txt -output pred.txt -gpu 0 --replace_unk

[2021-01-22 02:08:11,466 INFO] Translating shard 0.
/usr/local/lib/python3.6/dist-packages/onmt/translate/beam_search.py:275: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.6/dist-packages/onmt/translate/beam_search.py:275: UserWarning: An output with one or more elements was resized since it had shape [60], which does not match the required output shape [12, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an ou

In [57]:
translated_text = pd.read_csv('pred.txt', delimiter='\t', names=['translated_text'])

In [62]:
df['translated_body'] = translated_text

In [66]:
df['translated_body'] = df['translated_body'].astype(str)
df['translated_body'] = df['translated_body'].apply(lambda x: x.split(" "))
df['translated_body'] = df['translated_body'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

In [68]:
df.to_csv("processed_articles.csv", index=False)